# PCAI PDF RAG System

## Complete Implementation of 5 Required Components

This notebook demonstrates a working Retrieval-Augmented Generation system using PDF File

**Components Implemented:**
1. Knowledge Base (PDF)
2. Semantic Layer (sentence-transformers)
3. Retrieval System (FAISS)
4. Augmentation (context enrichment)
5. Generation (answer synthesis)

## Setup: Import All Components

Run this cell first to load all modules.

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers")

In [2]:
import sys, os
sys.path.append(os.path.abspath(".."))

In [3]:
# Import our 5 components
from knowledge_base import KnowledgeBase
from semantic_layer import SemanticLayer
from retrieval_system import RetrievalSystem
from augmentation import Augmentation
from generation import Generation
print("✅ All components imported successfully!")

✅ All components imported successfully!


## Component 1: Knowledge Base

Load PCAI dataset from pdf.

In [4]:

# Initialize KB with PDF and chunk settings
kb = KnowledgeBase(pdf_path="../data/hpe-pcai.pdf", chunk_size=200, chunk_overlap=50)

# Load PDF pages
documents = kb.load_pdf_data()

# Create sentence-preserving chunks
chunks = kb.create_chunks()

# Inspect stats
stats = kb.get_stats()
stats = kb.get_stats()
print(f"\n📊 Knowledge Base:")
print(f"   Pages: {stats['total_pages']}")
print(f"   Chunks: {stats['total_chunks']}")
print(f"   Source: {stats['source']}")
print(f"   Chunk Size       : {kb.chunk_size}")
print(f"   Chunk Overlap    : {kb.chunk_overlap}")

📚 Knowledge Base initialized

📥 Loading PDF: ../data/hpe-pcai.pdf
✅ Loaded 24 pages from PDF

 Creating chunks with LangChain RecursiveCharacterTextSplitter...
✅ Created 341 chunks

📊 Knowledge Base:
   Pages: 24
   Chunks: 341
   Source: ../data/hpe-pcai.pdf
   Chunk Size       : 200
   Chunk Overlap    : 50


## Component 2: Semantic Layer

Convert text to numerical embeddings for semantic comparison.

In [5]:
import random
import numpy as np

# =========================
# Random Preview: Chunks & Embeddings
# =========================

# Number of random samples to preview
N = 5

# Ensure embeddings exist
try:
    document_embeddings
except NameError:
    # Encode chunks if not done yet
    semantic = SemanticLayer()
    document_embeddings = semantic.encode_documents(kb.chunks)

# Random indices
random_indices = random.sample(range(len(kb.chunks)), min(N, len(kb.chunks)))

print(f"\n📄 Random preview of {N} chunks with embeddings:\n")

for i, idx in enumerate(random_indices):
    chunk = kb.chunks[idx]
    emb = document_embeddings[idx]
    
    print(f"--- Sample {i+1} (Index {idx}) ---")
    print(f"Page: {chunk['page']}")
    print(f"Text (first 200 chars): {chunk['chunk'][:200]}...")
    print(f"Embedding shape: {emb.shape}")
    print(f"Embedding sample (first 10 values): {emb[:10]}")
    print("-" * 70)


COMPONENT 2: Semantic Layer initializing...
   Loading model: all-MiniLM-L6-v2
   (First run downloads 22MB - takes 30 seconds)
✅ Model loaded! Embeddings dimension: 384

🔄 Encoding 341 document chunks to embeddings...


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

✅ Embeddings created: (341, 384)

📄 Random preview of 5 chunks with embeddings:

--- Sample 1 (Index 59) ---
Page: 4
Text (first 200 chars): HPE
 
Private Cloud AI
 helps organizations avoid the unpredictable costs
associated with public cloud AI services, especially for large-scale or continuous workloads....
Embedding shape: (384,)
Embedding sample (first 10 values): [-0.06213689 -0.00925779  0.06471906  0.02861802  0.0600124   0.01544559
 -0.04650822 -0.04205861  0.07658074 -0.01001094]
----------------------------------------------------------------------
--- Sample 2 (Index 334) ---
Page: 23
Text (first 200 chars): Create an 
HPE
 GreenLake workspace
Add Data Services Cloud Console to your workspace
Add the Private Cloud AI service to Data Services Cloud Console
Assign user roles
After the 
Private Cloud AI...
Embedding shape: (384,)
Embedding sample (first 10 values): [-0.02285308 -0.09133134  0.02503943 -0.07249123 -0.00623796  0.01931972
  0.00109714  0.05409911 -0.01556824  0.

## Component 3: Retrieval System

Build FAISS index for fast similarity search.

In [6]:
# Initialize Retrieval System
retrieval = RetrievalSystem(dimension=384)

# Build searchable index
retrieval.build_index(document_embeddings, kb.chunks)

print("\n✅ Retrieval system ready to search!")

🔍 COMPONENT 3: Retrieval System initialized (dim=384)

🔨 Building FAISS index...
   Documents: 341
   Embedding shape: (341, 384)
✅ Index built with 341 vectors

✅ Retrieval system ready to search!


In [7]:
# Inspect retrieval output
results = retrieval.search(semantic.encode_query("What are pcai core components"), top_k=3)
print("Results structure:\n")
for i, r in enumerate(results):
    print(f"Result {i}:")
    print(r)
    print("-" * 50)



🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents
Results structure:

Result 0:
{'rank': 1, 'document': {'page': 7, 'chunk': 'HPE\n AI Essentials Software.\nCore Components\nCore Components\nOpen-Source Tools & Frameworks\nOpen-Source Tools & Frameworks'}, 'similarity': 0.45020369059766646, 'distance': 1.2212167978286743}
--------------------------------------------------
Result 1:
{'rank': 2, 'document': {'page': 4, 'chunk': 'HPE\n AI Essentials\n with NVIDIA NIM™\nintegration, empowers organizations to swiftly build, deploy, and operate AI/ML models and GenAI applications. NVIDIA NIM™, part of'}, 'similarity': 0.44360095158939405, 'distance': 1.254278302192688}
--------------------------------------------------
Result 2:
{'rank': 3, 'document': {'page': 2, 'chunk': 'troubleshooting procedures, and maintenance protocols.\nPart Number: 20-PCAI-UG-ED1\nPublished: September 2025\nEdition: 1.5\n© Copyright 2025– Hewlett Packard Enterprise Development LP\nNotices\nNotice

## Test the Complete RAG Pipeline

Now let's ask a question and see all 5 components work together!

In [8]:
def ask_question(question, top_k=3):
    """Complete RAG pipeline for a question using PCAI PDF knowledge base"""
    
    print("=" * 70)
    print(f"❓ QUESTION: {question}")
    print("=" * 70)
    
    # COMPONENT 2: Encode query
    print("\n[Step 1] Encoding question...")
    query_embedding = semantic.encode_query(question)
    
    # COMPONENT 3: Retrieve relevant chunks
    print("[Step 2] Searching knowledge base...")
    results = retrieval.search(query_embedding, top_k=top_k)
    
    print(f"\n📚 Retrieved Top {top_k} Chunks:")
    for r in results:
        doc_page = r['document']['page']
        doc_text = r['document']['chunk']
        print(f"  • Page {doc_page} (similarity: {r['similarity']:.1%})")
        print(f"    Text snippet: {doc_text[:200]}...\n")
    
    # COMPONENT 4: Augment with context
    print("[Step 3] Creating enriched context...")
    augmentor = Augmentation()
    enriched = augmentor.create_context(question, results)
    
    # COMPONENT 5: Generate answer
    print("[Step 4] Generating answer...")
    generator = Generation(model_name='llama3.2:3b', use_llm=True)
    answer = generator.generate(enriched, results)
    
    # Display answer
    print("\n" + "=" * 70)
    print("💬 ANSWER:")
    print("=" * 70)
    print(answer)
    print("=" * 70)
    
    return answer

# Example usage
ask_question("What are pcai core tools")



❓ QUESTION: What are pcai core tools

[Step 1] Encoding question...
[Step 2] Searching knowledge base...

🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents

📚 Retrieved Top 3 Chunks:
  • Page 7 (similarity: 50.2%)
    Text snippet: HPE
 AI Essentials Software.
Core Components
Core Components
Open-Source Tools & Frameworks
Open-Source Tools & Frameworks...

  • Page 8 (similarity: 47.6%)
    Text snippet: AI assistants for the fastest path to inference with unparalleled performance.
HPE
 AI Essentials Software
 
currently includes the following pre-packaged NVIDIA NIM:...

  • Page 4 (similarity: 47.3%)
    Text snippet: HPE
 AI Essentials
 with NVIDIA NIM™
integration, empowers organizations to swiftly build, deploy, and operate AI/ML models and GenAI applications. NVIDIA NIM™, part of...

[Step 3] Creating enriched context...
⚡ COMPONENT 4: Augmentation initialized

⚡ Creating enriched context with 3 documents...
✅ Enriched context created (673 characters)
[Step 4] G

'Based on the provided context, PCAI Core Tools are not explicitly mentioned in the text. However, it can be inferred that "NVIDIA NIM" (which is a pre-packaged tool) might be related to "PCAi" as part of HPE\'s AI Essentials Software.\n\nSince there is no direct mention of "PCAi Core Tools," I\'ll provide an answer based on the context:\n\nIt appears that PCAI refers to NVIDIA\'s Model Optimizing Tool, which is mentioned in the text as a pre-packaged tool within HPE\'s AI Essentials Software. The specific tools or frameworks included are not explicitly stated, but it can be inferred that they might include Open-Source Tools & Frameworks.\n\nIf you\'re looking for more information on PCAi Core Tools, I recommend checking other sources or documentation related to NVIDIA\'s Model Optimizing Tool and its features.\n\n======================================================================\n📚 SOURCES:\n======================================================================\n\n[1] Page 7 (Simi

## Try More Questions!

Test the system with different queries.

In [9]:
# Try these questions or write your own!

ask_question("What is pcai engineered system for?")

❓ QUESTION: What is pcai engineered system for?

[Step 1] Encoding question...
[Step 2] Searching knowledge base...

🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents

📚 Retrieved Top 3 Chunks:
  • Page 5 (similarity: 48.1%)
    Text snippet: HPE
 offers comprehensive infrastructure solutions, while NVIDIA
provides support for its AI frameworks and GPU technologies, enabling organizations to build and deploy powerful AI systems on-...

  • Page 8 (similarity: 47.8%)
    Text snippet: NVIDIA Blueprints
NVIDIA Blueprints
.
You can use Blueprints to jumpstart or accelerate the development and deployment of AI use cases. Blueprints cover a wide range of use...

  • Page 4 (similarity: 47.4%)
    Text snippet: HPE
 AI Essentials
 with NVIDIA NIM™
integration, empowers organizations to swiftly build, deploy, and operate AI/ML models and GenAI applications. NVIDIA NIM™, part of...

[Step 3] Creating enriched context...
⚡ COMPONENT 4: Augmentation initialized

⚡ Creating enri

'Based on the provided context, it appears that PCAI (not explicitly mentioned in the PDF) is not a clear topic. However, I can infer that PCAI might be related to AI systems.\n\nAccording to Page 4, HPE offers "AI Essentials" with NVIDIA NIM integration, which enables organizations to build, deploy, and operate AI/ML models and GenAI applications.\n\nConsidering this information, it\'s possible that PCAI is an engineered system for building and deploying AI use cases. However, without explicit mention of PCAI in the PDF, I couldn\'t determine its exact purpose or definition.\n\n======================================================================\n📚 SOURCES:\n======================================================================\n\n[1] Page 5 (Similarity: 48.1%)\n    Preview: HPE\n offers comprehensive infrastructure solutions, while NVIDIA\nprovides support for its AI frameworks and GPU technologies, enabling organizations t...\n\n[2] Page 8 (Similarity: 47.8%)\n    Preview: NVIDIA 

In [10]:
ask_question("does HPE offer supportf or AI products on delivery")

❓ QUESTION: does HPE offer supportf or AI products on delivery

[Step 1] Encoding question...
[Step 2] Searching knowledge base...

🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents

📚 Retrieved Top 3 Chunks:
  • Page 4 (similarity: 60.1%)
    Text snippet: Hewlett Packard Enterprise
 features explicit support for 
HPE
 and NVIDIA AI software, and is...

  • Page 7 (similarity: 58.3%)
    Text snippet: production. 
HPE
 AI Essentials Software provides a ready-to-run set of AI and open-source tools to build end-to-end GenAI solutions.
From novice to expert, 
HPE...

  • Page 7 (similarity: 58.2%)
    Text snippet: HPE
 AI Essentials Software is part of the 
HPE
 Private Cloud AI software and data foundation layer that accelerates the time from AI pilot to
production. 
HPE...

[Step 3] Creating enriched context...
⚡ COMPONENT 4: Augmentation initialized

⚡ Creating enriched context with 3 documents...
✅ Enriched context created (658 characters)
[Step 4] Generating answe

'Yes, HPE offers support for AI products on delivery. The relevant text states:\n\n"Hewlett Packard Enterprise features explicit support for HPE and NVIDIA AI software..."\n\nThis indicates that HPE provides support for its own AI software, as well as software from NVIDIA. Additionally, the text mentions that HPE\'s AI Essentials Software is part of a private cloud AI solution that includes "explicit support" for AI products.\n\nTherefore, it can be concluded that HPE does offer support for AI products on delivery.\n\n======================================================================\n📚 SOURCES:\n======================================================================\n\n[1] Page 4 (Similarity: 60.1%)\n    Preview: Hewlett Packard Enterprise\n features explicit support for \nHPE\n and NVIDIA AI software, and is\n\n[2] Page 7 (Similarity: 58.3%)\n    Preview: production. \nHPE\n AI Essentials Software provides a ready-to-run set of AI and open-source tools to build end-to-end GenAI so

In [11]:
# Your own question here!
ask_question("Who is customer support engineer, doe they helo with AI platform")

❓ QUESTION: Who is customer support engineer, doe they helo with AI platform

[Step 1] Encoding question...
[Step 2] Searching knowledge base...

🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents

📚 Retrieved Top 3 Chunks:
  • Page 5 (similarity: 49.4%)
    Text snippet: Private Cloud AI Stack
Private Cloud AI Stack
Developer System
Developer System
HPE AI Essentials: Core Concepts
HPE AI Essentials: Core Concepts
Solution components
Solution components...

  • Page 23 (similarity: 48.8%)
    Text snippet: Maintain self-service access to various AI tools, frameworks, and development environments, such as 
HPE
 AI Essentials Software...

  • Page 4 (similarity: 48.6%)
    Text snippet: support, trusted AI services for data and model compliance, and features ensuring AI pipeline compliance, explainability, and reproducibility
throughout the AI lifecycle, 
HPE
 AI Essentials...

[Step 3] Creating enriched context...
⚡ COMPONENT 4: Augmentation initialized

⚡ Creating enr

'Based on the provided context, a Customer Support Engineer may not necessarily be directly involved with an AI platform.\n\nHowever, they might be responsible for supporting customers who are using HPE AI Essentials Software, which includes features such as maintaining self-service access to various AI tools, frameworks, and development environments (Page 23). Additionally, they may provide support related to trusted AI services, data and model compliance, explainability, and reproducibility throughout the AI lifecycle (Page 4).\n\nThe connection to an AI platform is not explicitly stated in the provided context. It seems that Customer Support Engineers might be more focused on supporting customers using HPE AI Essentials Software, rather than specifically working with AI platforms.\n\n======================================================================\n📚 SOURCES:\n======================================================================\n\n[1] Page 5 (Similarity: 49.4%)\n    Preview:

## Interactive Mode

Keep asking questions!

In [ ]:
# Interactive questioning
while True:
    user_question = input("\nYour question (or 'quit' to stop): ").strip()
    
    if user_question.lower() in ['quit', 'exit', 'q']:
        print("👋 PDF RAG here to answer")
        break
    
    if user_question:
        ask_question(user_question)
    else:
        print("Please enter a question!")


Your question (or 'quit' to stop):  what is the optimum environment recommended


❓ QUESTION: what is the optimum environment recommended

[Step 1] Encoding question...
[Step 2] Searching knowledge base...

🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents

📚 Retrieved Top 3 Chunks:
  • Page 11 (similarity: 78.0%)
    Text snippet: Optimum environment
Optimum environment...

  • Page 11 (similarity: 52.4%)
    Text snippet: Optimum environment
Optimum environment
To provide optimum performance with minimum maintenance for your rack environment, you must meet specific requirements for airflow,...

  • Page 3 (similarity: 49.0%)
    Text snippet: Solution components
Optimum environment
Airflow requirements
Power requirements
Space requirements
Temperature requirements
Firewall and port requirements
Networking requirements...

[Step 3] Creating enriched context...
⚡ COMPONENT 4: Augmentation initialized

⚡ Creating enriched context with 3 documents...
✅ Enriched context created (626 characters)
[Step 4] Generating answer...
🤖 COMPONENT 5: Generation i


Your question (or 'quit' to stop):  do i need a spearate nvidia subscription or the pcai comes with it?


❓ QUESTION: do i need a spearate nvidia subscription or the pcai comes with it?

[Step 1] Encoding question...
[Step 2] Searching knowledge base...

🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents

📚 Retrieved Top 3 Chunks:
  • Page 17 (similarity: 57.6%)
    Text snippet: Activating the NVIDIA software subscription
The following table describes adding the NVIDIA license information for various Private Cloud AI configurations....

  • Page 17 (similarity: 54.7%)
    Text snippet: subscription
.
See 
Activating the NVIDIA software
subscription
.
HPE
HPE
 Private Cloud AI Platform
 Private Cloud AI Platform
The subscription is provided through
GreenLake Cloud....

  • Page 18 (similarity: 54.1%)
    Text snippet: 4
. 
The customer must register on the
NVIDIA website to receive NVIDIA
support.
5
. 
No additional on-prem licensing is
required.
Developer and Large Configuration (NVIDIA...

[Step 3] Creating enriched context...
⚡ COMPONENT 4: Augmentation initialized

⚡ C


Your question (or 'quit' to stop):  what are the licensing reqs


❓ QUESTION: what are the licensing reqs

[Step 1] Encoding question...
[Step 2] Searching knowledge base...

🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents

📚 Retrieved Top 3 Chunks:
  • Page 15 (similarity: 53.9%)
    Text snippet: Licensing requirements
Licensing requirements
The following table describes how you receive and apply license information for the software components that make up the 
HPE...

  • Page 11 (similarity: 49.0%)
    Text snippet: Subtopics
Optimum environment
Optimum environment
Licensing requirements
Licensing requirements
Activating the NVIDIA software subscription
Activating the NVIDIA software subscription...

  • Page 16 (similarity: 47.8%)
    Text snippet: Product
How to receive the license information
How to receive the license information
How to apply the license information
How to apply the license information...

[Step 3] Creating enriched context...
⚡ COMPONENT 4: Augmentation initialized

⚡ Creating enriched context with 3 docu